In [1]:
using JuMP
using GLPKMathProgInterface

Pkg.add("DataDeps")

 Resolving package versions...
  Updating `~/.julia/Project.toml`
 [no changes]
  Updating `~/.julia/Manifest.toml`
 [no changes]


In [2]:
Pkg.add("ExcelFiles")


 Resolving package versions...
  Updating `~/.julia/Project.toml`
 [no changes]
  Updating `~/.julia/Manifest.toml`
 [no changes]


In [3]:
Pkg.add("DataFrames")

 Resolving package versions...
  Updating `~/.julia/Project.toml`
 [no changes]
  Updating `~/.julia/Manifest.toml`
 [no changes]


In [4]:
using DataDeps

register(DataDep(
    "AUSNUT Food Nutrient Database",
    """
        Website: http://www.foodstandards.gov.au/science/monitoringnutrients/
        Excel spreedsheet of food nutrient values prepared by Food Standard Australia New Zealand, between 2011 and 2013.
    """,
    "http://www.foodstandards.gov.au/science/monitoringnutrients/ausnut/Documents/8b.%20AUSNUT%202011-13%20AHS%20Food%20Nutrient%20Database.xls",
     "4bac90de8ab2673a2f9b19f0be4c822aac1832ed8314117666e5464ed57cb544"
        ;
    post_fetch_method = (fn)->mv(fn, "database.xls")
));


In [5]:


using ExcelFiles, DataFrames

const nutrients = DataFrame(load("ausnut.xls", "Food Nutrient Database"))
nutrients[1:4,:]



,Food ID,Survey ID,Food Name,Survey flag,"Energy, with dietary fibre (kJ)","Energy, without dietary fibre (kJ)",Moisture (g),Protein (g),Total fat (g),"Available carbohydrates, with sugar alcohols (g)","Available carbohydrates, without sugar alcohol (g)",Starch (g),Total sugars (g),Added sugars (g),Free sugars (g),Dietary fibre (g),Alcohol (g),Ash (g),Preformed vitamin A (retinol) (µg),Beta-carotene (µg),Provitamin A (b-carotene equivalents) (µg),Vitamin A retinol equivalents (µg),Thiamin (B1) (mg),Riboflavin (B2) (mg),Niacin (B3) (mg),Niacin derived equivalents (mg),"Folate, natural (µg)",Folic acid (µg),Total Folates (µg),Dietary folate equivalents (µg),Vitamin B6 (mg),Vitamin B12 (µg),Vitamin C (mg),Alpha-tocopherol (mg),Vitamin E (mg),Calcium (Ca) (mg),Iodine (I) (µg),Iron (Fe) (mg),Magnesium (Mg) (mg),Phosphorus (P) (mg),Potassium (K) (mg),Selenium (Se) (µg),Sodium (Na) (mg),Zinc (Zn) (mg),Caffeine (mg),Cholesterol (mg),Tryptophan (mg),Total saturated fat (g),Total monounsaturated fat (g),Total polyunsaturated fat (g),Linoleic acid (g),Alpha-linolenic acid (g),C20:5w3 Eicosapentaenoic (mg),C22:5w3 Docosapentaenoic (mg),C22:6w3 Docosahexaenoic (mg),Total long chain omega 3 fatty acids (mg),Total trans fatty acids (mg)
,String,Float64,String,Any,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,10F40019,3.1103e7,"Beef, extract, bonox",missing,401.0,401.0,56.6,16.6,0.2,6.5,6.5,6.5,0.0,0.0,0.0,0.0,0.0,19.8,0.0,0.0,0.0,0.0,0.36,0.27,5.4,7.71,6.0,0.0,6.0,6.0,0.23,8.0,0.0,0.6,0.6,110.0,9.1,2.0,60.0,360.0,690.0,4.0,6660.0,1.5,0.0,0.0,136.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,13A12001,3.1302e7,"Basil, dried",missing,1079.0,774.0,10.0,18.2,5.5,15.5,15.5,15.5,0.0,0.0,0.0,38.2,0.0,15.5,0.0,27135.0,27334.0,4556.0,0.327,1.9,7.79,11.62,436.0,0.0,436.0,436.0,1.34,0.0,337.0,6.9,6.91,2091.0,57.0,16.36,273.0,291.0,3818.0,2.7,118.0,17.27,0.0,0.0,225.0,2.36,1.4,0.61,0.26,0.34,0.0,0.0,0.0,0.0,0.0
3,10E10113,3.1302e7,"Cardamom, seeds, ground",missing,1333.0,1109.0,8.3,10.8,6.7,40.5,40.5,31.0,9.5,0.0,0.0,28.0,0.0,5.8,0.0,0.0,0.0,0.0,0.198,0.182,1.1,3.74,0.0,0.0,0.0,0.0,0.23,0.0,21.0,0.5,0.5,383.0,0.5,13.97,229.0,178.0,1119.0,0.5,18.0,7.47,0.0,0.0,155.0,2.2,2.81,1.39,1.0,0.39,0.0,0.0,0.0,0.0,0.0
4,10E10098,3.1302e7,Chilli (chili) powder,missing,1441.0,1167.0,7.8,12.3,16.8,20.5,20.5,10.4,10.1,0.0,0.0,34.2,0.0,11.8,0.0,15000.0,17790.0,2965.0,0.35,0.79,7.9,8.98,9.0,0.0,9.0,9.0,2.09,0.0,0.0,38.1,38.14,278.0,0.5,14.3,170.0,303.0,1920.0,20.4,1010.0,2.7,0.0,0.0,64.0,2.41,3.15,7.85,7.32,0.52,0.0,0.0,0.0,0.0,0.0


In [6]:
#nutrients[:, Symbol("Protein (g)")] 

In [7]:
#nutrients[1, Symbol("Protein (g)")]



In [8]:
using JuMP
using LinearAlgebra
function new_diet(solver = GLPKSolverLP())
    m = Model(solver= solver)
    @variable(m, diet[1:size(nutrients,1)], lowerbound=0)
    m, diet 
end



new_diet (generic function with 2 methods)

In [9]:
function total_intake(nutrient_name, diet)
    nutrients[:, Symbol(nutrient_name)] ⋅ diet
end

function basic_nutrient_requirements(m::Model, diet, weight = 62) # average human is 62 kg
    intake(nutrient_name) = total_intake(nutrient_name, diet) 
    # ^ closure over the diet given as an argument
        
    @constraints(m, begin
        70 <= intake("Protein (g)") 
        70 <= intake("Total fat (g)") 
        24 <= intake("Total saturated fat (g)") 
        90 <= intake("Total sugars (g)") 
        intake("Alcohol (g)") <= 100
        310 <= intake("Available carbohydrates, with sugar alcohols (g)") 
        intake("Sodium (Na) (mg)") <= 2300 # Heart Council
        intake("Caffeine (mg)") <= 400
        25 <=intake("Dietary fibre (g)") 
        24 <= intake("Total trans fatty acids (mg)") 
        25 <= intake("Vitamin A retinol equivalents (µg)") <= 900
        1.2 <= intake("Thiamin (B1) (mg)") 
        400 <= intake("Dietary folate equivalents  (µg)") 
        1000 <= intake("Folic acid  (µg)") # https://ods.od.nih.gov/factsheets/Folate-HealthProfessional/
        75 <= intake("Vitamin C (mg)") <= 2000 ##https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        95 <= intake("Iodine (I) (µg)") <=1100 ##https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        6 <= intake("Iron (Fe) (mg)") <= 45 # #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        330 <= intake("Magnesium (Mg) (mg)") 
        580 <= intake("Phosphorus (P) (mg)") <= 4000 #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        4700 <= intake("Potassium (K) (mg)") 
        800 <= intake("Calcium (Ca) (mg)") <= 2500 #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        3.5*weight <= intake("Tryptophan (mg)") <= 0.5*weight*1600 #0.5 * LD50 https://books.google.com.au/books?id=he7LBQAAQBAJ&pg=PA210&lpg=PA210&dq=Tryptophan+LD-50&source=bl&ots=yzi5Hx7hM2&sig=sd78LGAnavIwtT5AP-L4fzsFLZM&hl=en&sa=X&ved=0ahUKEwjJhcjFn6rQAhUKkJQKHbw6AiwQ6AEIMTAE#v=onepage&q=Tryptophan%20LD-50&f=false
                                                #http://www.livestrong.com/article/541961-how-much-tryptophan-per-day/
        17 <= intake("Linoleic acid (g)") 

        45 <= intake("Selenium (Se) (µg)") <= 400 #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        11 <= intake("Zinc (Zn) (mg)") <= 40 #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
            
            
        6 <= intake("Vitamin E (mg)") <= 1_200    #https://www.livestrong.com/article/465706-vitamin-e-overdose-levels/
            
        1.3 <= intake("Vitamin B6 (mg)") <= 300  #https://www.news-medical.net/health/Can-You-Take-Too-Much-Vitamin-B.aspx
        2.4 <= intake("Vitamin B12  (µg)")   #https://www.news-medical.net/health/Can-You-Take-Too-Much-Vitamin-B.aspx
            
    end)
    
end



basic_nutrient_requirements (generic function with 2 methods)

In [10]:


filter(x->occursin(x, "Vitamin B"), String.(collect(names(nutrients))))



0-element Array{String,1}

In [11]:
function show_diet(diet)
    for ii in eachindex(diet)
        amount = getvalue(diet[ii])
        if amount > 0
            name = nutrients[ii, Symbol("Food Name")]
            grams = round(Int, 100amount) # amount is expressed in units of 100 grams
            display_as = grams < 1  ? ">0" : string(grams) #Make things that are "0.001 grams" show as ">0 grams"
            println(lpad(display_as, 5, " "), " grams \t", name)
        end
    end
end



show_diet (generic function with 1 method)

# Dieta 1
## dieta zapewniająca minimalne potrzeby dla osoby ważącej 62 kg


In [12]:


m, diet = new_diet()
basic_nutrient_requirements(m, diet);

@constraint(m, diet .<= 5) # nothing more than 500g
@constraint(m, total_intake("Moisture (g)", diet) <= 0.25sum(diet)*100) # Less than 25% liquid

@objective(m, Min,  
    total_intake("Energy, with dietary fibre (kJ)", diet) + 0.01sum(diet)
)
@show status = solve(m)
show_diet(diet)



status = solve(m) = :Optimal
  500 grams 	Salt substitute, potassium chloride
    1 grams 	Salt, flavoured
  500 grams 	Cream of tartar, dry powder
    2 grams 	Caffeine
   13 grams 	Calcium
    8 grams 	Fibre
  500 grams 	Folic acid
    9 grams 	Iodine
  500 grams 	Thiamin
   19 grams 	Vitamin C
   12 grams 	Vitamin E
  124 grams 	Coffee, macchiato, from ground coffee beans, with skim cows milk
  144 grams 	Tea, regular, white, brewed from leaf or teabags, with cows milk not further defined
   19 grams 	Oil, safflower
    3 grams 	Shortening, commercial, vegetable fat (for coatings, creams, icings, confectionery & fillings)
  291 grams 	Octopus, cooked, with or without added fat
    2 grams 	Oyster, cooked, with or without added fat
  152 grams 	Bar, nougat & caramel centre, milk chocolate-coated
  217 grams 	Lolly, non-mint flavours, intense sweetened
  167 grams 	Bok choy or choy sum, baked, roasted, fried, stir-fried, grilled or BBQ'd, no added fat
  229 grams 	Mixed vegetables, As

In [13]:
using DelimitedFiles

function forbid_liquids(m::Model, diet, keywords=["milk", "drink", "Water", "Milk"])
    # not blocking lowercase "water" as that would block "Soup made with water"
    
    inds = keyword_using_inds = findall(x->any(occursin.(x, keywords)), nutrients[:, Symbol("Food Name")])
    high_moisture_inds =  findall(x->x>50, nutrients[:, Symbol("Moisture (g)")])
    inds = intersect(keyword_using_inds, high_moisture_inds)
    #@show nutrients[inds, Symbol("Food Name")]
    @constraint(m, diet[inds] .== 0 ) # nothing more than 500g

end



forbid_liquids (generic function with 2 methods)

# Dieta 1 z ograniczeniem płynów

In [14]:


m, diet = new_diet()
basic_nutrient_requirements(m, diet);
forbid_liquids(m, diet)

@constraint(m, diet .<= 3) # nothing more than 300g

@objective(m, Min,  
    total_intake("Energy, with dietary fibre (kJ)", diet) + 0.01sum(diet)
)
@show status = solve(m)
show_diet(diet)



status = solve(m) = :Optimal
  300 grams 	Stock, liquid, all flavours (except fish), homemade from basic ingredients
  111 grams 	Stock, liquid, fish, homemade from basic ingredients
   10 grams 	Folic acid
   >0 grams 	Thiamin
  300 grams 	Coffee, from ground coffee beans, with regular fat coffee whitener
  300 grams 	Coffee, cappuccino, from ground coffee beans, with skim cows milk
  300 grams 	Coffee, macchiato, from ground coffee beans, decaffeinated, with regular fat cows milk
   48 grams 	Coffee, long black style, from ground coffee beans, double shot, without milk
  300 grams 	Tea, regular, white, brewed from leaf or teabags, with cows milk not further defined
  300 grams 	Water, bottled, with added sugar, vitamins & minerals
  300 grams 	Milk, almond, fluid
  300 grams 	Milk, cow, fluid, prepared from dry powder, regular fat, standard dilution
  300 grams 	Milk & water, reduced fat cow's milk & tap water
  300 grams 	Milk & water, skim cow's milk & tap water
   15 grams 	Oil, s

# Dieta 2
## Dieta dla sportowca wytrzymałościowego ważącego 70 kg
### W diecie uwzględniona została wysoka podaż białka

In [15]:
#based on articles from https://www.nal.usda.gov/fnic/dri-tables-and-application-reports
#recommendation for man 19-30 years old
function endurance_athlete_requirements(m::Model, diet, weight = 70) # his weight is 70 kg
    intake(nutrient_name) = total_intake(nutrient_name, diet) 
    # ^ closure over the diet given as an argument
        
    @constraints(m, begin
        91 <= intake("Protein (g)") 
        70 <= intake("Total fat (g)") 
        24 <= intake("Total saturated fat (g)") 
        70 <= intake("Total sugars (g)") 
        intake("Alcohol (g)") <= 0
        350 <= intake("Available carbohydrates, with sugar alcohols (g)") 
        intake("Sodium (Na) (mg)") <= 2300 # Heart Council
        intake("Caffeine (mg)") <= 400
        25 <=intake("Dietary fibre (g)") 
        24 <= intake("Total trans fatty acids (mg)") 
        900 <= intake("Vitamin A retinol equivalents (µg)") <= 3000
        1.2 <= intake("Thiamin (B1) (mg)") 
        400 <= intake("Dietary folate equivalents  (µg)") 
        1000 <= intake("Folic acid  (µg)") # https://ods.od.nih.gov/factsheets/Folate-HealthProfessional/
        75 <= intake("Vitamin C (mg)") <= 2000 ##https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        #600 <= intake("Vitamin D (IU)") <= 4000
        95 <= intake("Iodine (I) (µg)") <=1100 ##https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        6 <= intake("Iron (Fe) (mg)") <= 45 # #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        330 <= intake("Magnesium (Mg) (mg)") 
        580 <= intake("Phosphorus (P) (mg)") <= 4000 #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        4700 <= intake("Potassium (K) (mg)") 
        800 <= intake("Calcium (Ca) (mg)") <= 2500 #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        3.5*weight <= intake("Tryptophan (mg)") <= 0.5*weight*1600 #0.5 * LD50 https://books.google.com.au/books?id=he7LBQAAQBAJ&pg=PA210&lpg=PA210&dq=Tryptophan+LD-50&source=bl&ots=yzi5Hx7hM2&sig=sd78LGAnavIwtT5AP-L4fzsFLZM&hl=en&sa=X&ved=0ahUKEwjJhcjFn6rQAhUKkJQKHbw6AiwQ6AEIMTAE#v=onepage&q=Tryptophan%20LD-50&f=false
                                                #http://www.livestrong.com/article/541961-how-much-tryptophan-per-day/
        17 <= intake("Linoleic acid (g)")
        #550 <= intake("Choline (g)")
        #16 <= intake("Niacin (g)")
        #5 <= intake("Patothenic Acid (mg)")
        
        45 <= intake("Selenium (Se) (µg)") <= 400 #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        11 <= intake("Zinc (Zn) (mg)") <= 40 #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
            
            
        15 <= intake("Vitamin E (mg)") <= 1000    #https://www.livestrong.com/article/465706-vitamin-e-overdose-levels/
        #120 <= intake("Vitamin K (µg)") 
            
        1.3 <= intake("Vitamin B6 (mg)") <= 100  #https://www.news-medical.net/health/Can-You-Take-Too-Much-Vitamin-B.aspx
        2.4 <= intake("Vitamin B12  (µg)")   #https://www.news-medical.net/health/Can-You-Take-Too-Much-Vitamin-B.aspx
        #1.3 <= intake("Vitamin B2 (mg)")
        
    end)
    
end





endurance_athlete_requirements (generic function with 2 methods)

In [16]:


m, diet = new_diet()
endurance_athlete_requirements(m, diet);
forbid_liquids(m, diet)
@constraint(m, diet .<= 3) # nothing more than 300g


@constraint(m,
    9_000 <=
        total_intake("Energy, with dietary fibre (kJ)", diet)
    <= 12_000
    )
@objective(m, Max,  
    total_intake("Protein (g)", diet)
)
@show status = solve(m)
show_diet(diet)



status = solve(m) = :Optimal
   10 grams 	Folic acid
  300 grams 	Coffee, from ground coffee beans, with regular fat coffee whitener
  300 grams 	Coffee, cappuccino, from ground coffee beans, with skim cows milk
  300 grams 	Coffee, macchiato, from ground coffee beans, decaffeinated, with regular fat cows milk
   49 grams 	Coffee, long black style, from ground coffee beans, double shot, without milk
  300 grams 	Tea, regular, white, brewed from leaf or teabags, with cows milk not further defined
  224 grams 	Water, bottled, with added sugar, vitamins & minerals
   26 grams 	Milk, almond, fluid
  300 grams 	Milk, cow, fluid, prepared from dry powder, regular fat, standard dilution
  300 grams 	Milk & water, reduced fat cow's milk & tap water
  300 grams 	Milk & water, skim cow's milk & tap water
   20 grams 	Oil, safflower
  300 grams 	Beef, rump steak, fully-trimmed, raw
   28 grams 	Veal, stir-fry strips, untrimmed, raw
  300 grams 	Pork, silverside steak, untrimmed, raw
  300 grams 	

# Dieta 3
## Dieta dla grupy 20 osób zapewniająca wymagane minimum wartości odżywczych oraz niską wagę
### Może być zastosowana np. w podróży po schroniskach

In [17]:
function total_intake(nutrient_name, diet)
    nutrients[:, Symbol(nutrient_name)] ⋅ diet
end

function basic_nutrient_group(m::Model, diet, weight = 62) # average human is 62 kg
    intake(nutrient_name) = total_intake(nutrient_name, diet) 
    # ^ closure over the diet given as an argument
        
    @constraints(m, begin
        70 <= intake("Protein (g)") 
        70 <= intake("Total fat (g)") 
        24 <= intake("Total saturated fat (g)") 
        90 <= intake("Total sugars (g)") 
        intake("Alcohol (g)") <= 100
        310 <= intake("Available carbohydrates, with sugar alcohols (g)") 
        intake("Sodium (Na) (mg)") <= 2300 # Heart Council
        intake("Caffeine (mg)") <= 400
        25 <=intake("Dietary fibre (g)") 
        24 <= intake("Total trans fatty acids (mg)") 
        25 <= intake("Vitamin A retinol equivalents (µg)") <= 900
        1.2 <= intake("Thiamin (B1) (mg)") 
        400 <= intake("Dietary folate equivalents  (µg)") 
        1000 <= intake("Folic acid  (µg)") # https://ods.od.nih.gov/factsheets/Folate-HealthProfessional/
        75 <= intake("Vitamin C (mg)") <= 2000 ##https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        95 <= intake("Iodine (I) (µg)") <=1100 ##https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        6 <= intake("Iron (Fe) (mg)") <= 45 # #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        330 <= intake("Magnesium (Mg) (mg)") 
        580 <= intake("Phosphorus (P) (mg)") <= 4000 #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        4700 <= intake("Potassium (K) (mg)") 
        800 <= intake("Calcium (Ca) (mg)") <= 2500 #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        3.5*weight <= intake("Tryptophan (mg)") <= 0.5*weight*1600 #0.5 * LD50 https://books.google.com.au/books?id=he7LBQAAQBAJ&pg=PA210&lpg=PA210&dq=Tryptophan+LD-50&source=bl&ots=yzi5Hx7hM2&sig=sd78LGAnavIwtT5AP-L4fzsFLZM&hl=en&sa=X&ved=0ahUKEwjJhcjFn6rQAhUKkJQKHbw6AiwQ6AEIMTAE#v=onepage&q=Tryptophan%20LD-50&f=false
                                                #http://www.livestrong.com/article/541961-how-much-tryptophan-per-day/
        17 <= intake("Linoleic acid (g)") 

        45 <= intake("Selenium (Se) (µg)") <= 400 #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        11 <= intake("Zinc (Zn) (mg)") <= 40 #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
            
            
        6 <= intake("Vitamin E (mg)") <= 1_200    #https://www.livestrong.com/article/465706-vitamin-e-overdose-levels/
            
        1.3 <= intake("Vitamin B6 (mg)") <= 300  #https://www.news-medical.net/health/Can-You-Take-Too-Much-Vitamin-B.aspx
        2.4 <= intake("Vitamin B12  (µg)")   #https://www.news-medical.net/health/Can-You-Take-Too-Much-Vitamin-B.aspx
            
    end)
    
end



basic_nutrient_group (generic function with 2 methods)

In [19]:


m, diet = new_diet()
basic_nutrient_group(m, diet);
forbid_liquids(m, diet)

@constraint(m, diet .<= 5) # nothing more than 500g
@constraint(m, total_intake("Moisture (g)", diet) <= 0.25sum(diet)*100) # Less than 25% liquid

# @objective(m, Min,  
#      total_intake("Energy, with dietary fibre (kJ)", diet) + 0.01sum(diet)
#  )
@objective(m, Min, sum(diet))
@show status = solve(m)
show_diet(diet*20)

println("Total: $(round(Int, 100sum(getvalue(diet)))) grams")

status = solve(m) = :Optimal
  106 grams 	Salt substitute, potassium chloride
  204 grams 	Fibre
 3407 grams 	Biscuit, savoury, rice cracker, from brown rice, all flavours
 1957 grams 	Breakfast cereal, mixed grain (rice & wheat), flakes, added vitamins B1, B2, B3, B6 & folate, Ca, Fe & Zn
  224 grams 	Oil, copha
  209 grams 	Oil, safflower
   11 grams 	Nut, brazil, with or without skin, raw, unsalted
  501 grams 	Protein powder, protein 45%, reduced sugars, fortified
   48 grams 	Protein powder, protein 20%, fortified, caffeinated
 1045 grams 	Potato crisps or chips, plain, unsalted
  737 grams 	Intense sweetener, containing aspartame/acesulfame-potassium, tablet
 1151 grams 	Sugar, white, with added stevia, granulated
   56 grams 	Seaweed, nori, dried
Total: 483 grams
